In [15]:
import ROOT
import numpy as np
from tqdm.notebook import tqdm, trange
from time import sleep
import toml

%jsroot on

In [18]:
config = toml.load("stage0_config.toml")

stage = config["general"]["stage"]
numch = config["general"]["numch"]
print(numch)

32


In [2]:
# hist = ROOT.TH1D("test","Test", 10, -3, 3)
# hist.FillRandom("gaus",10000)
# hist.Draw()
# ROOT.gROOT.GetListOfCanvases().Draw()

In [2]:
def tof_1mus_to_En(TOF: any, L: float) -> any:
    """Converts TOF in mu s to neutron energy in eV using the length from moderator to target L in m"""

    return ((72.3 * L)/ TOF)**2

def TOF_10ns_to_En(TOF: any, L: float) -> any:
    """Converts TOF in units of 10 ns to neutron energy in eV using the length from moderator to target L in m"""

    return ((72.3 * L)/ (TOF/100))**2

In [4]:
# TOF_10ns_to_En(48000,21.5)

In [3]:
xbins_tof = np.linspace(1,100000+2,100000+1)
xbins_En=TOF_10ns_to_En(xbins_tof,21.5)
xbins_En=np.sort(xbins_En)

In [6]:
# xbins = np.linspace(-3,3,10+1)
# print(xbins)
# hist1 = ROOT.TH1D("test1","Test", 10, xbins)
# hist1.FillRandom("gaus",10000)
# hist1.Draw()
# ROOT.gROOT.GetListOfCanvases().Draw()

In [7]:
# ROOT.EnableImplicitMT()

# tree_name = "rawTree"
# # file_name = "/Volumes/WD_BLACK/2023Apr/Ge/rawroot/rawroot_run_001[4-9]*"
# file_name = "rawroot/rawroot*"

# df = ROOT.RDataFrame(tree_name, file_name)
# # print(df.GetColumnNames())
# # print(df.GetColumnType("tof"))

# df = df.Define("En","pow((72.3*21.5/(tof/100.0)),2)").Define("tof_mu","tof/100.0").Filter("detector == 1")
# df1 = df.Filter("detector == 1")
# df2 = df.Filter("detector == 2")


# h = df.Histo1D(hist_model,"En")
# h1 = df.Histo1D(("hEn1","hEn1;E_{n } [eV];counts",10000,0,100),"En")
# h1.SetLineColor(ROOT.kRed)
# # h2 = df2.Histo1D(("hEn2","hEn2;E_{n } [eV];counts",10000,0,100),"En")
# h2 = df.Histo1D(("tof","htof;10 ns;counts",100000,0,100000),"tof")
# # print(df.GetDefinedColumnNames())
# # print(df.GetColumnNames())

# # h.ResetStats()
# h.Draw()
# h1.Draw("hist same")
# # d1 = df.Display("TOF", 128)
# # h2.Draw()

# print(df.GetNRuns())
# # df.Describe().Print()
# # df.Filter("detector == 1").Display("tof").Print()
# # d1.Print()
# # print(df1.GetNRuns())
# # ROOT.gROOT.GetListOfCanvases().Draw()

In [8]:
# h2.Draw()
# # ROOT.gROOT.GetListOfCanvases().Draw()
# print(df1.GetNRuns())

In [12]:
ROOT.EnableImplicitMT()

numch = 32

bit_puflag=32768;
bit_foflag=65536;

calib = [1.1]*32

tree_name = "rawTree"
# file_name = "/Volumes/WD_BLACK/2023Apr/Ge/rawroot/rawroot_run_001[4-9]*"
file_name = "rawroot/rawroot*"

df = ROOT.RDataFrame(tree_name, file_name)


# df = (df.Define("En","pow((72.3*21.5/(tof/100.0)),2)")
#       .Define("tof_mus","tof/100.0")
#       .Define("Egam","PulseHeight"))

df = df.Define("En","pow((72.3*21.5/(tof/100.0)),2)").Define("tof_mus","tof/100.0")
# df = df.Range(100000)


df_pu = df.Filter(f"Flags == {bit_puflag}")  
df = df.Filter(f"Flags != {bit_puflag}")      


df_ch = []
df_ch_pu = []


hTOF = []
hEn = []
hPulseHeight = []
hEgam = []


hTOF_pu = []
hEn_pu = []
hPulseHeight_pu = []
hEgam_pu = []


hEn_model = []
hTOF_model = []
hPulseHeight_model = []
hEgam_model = []


for ch in range(numch):
    hTOF_model.append(ROOT.RDF.TH1DModel(f"hTOF_d{ch}",f"hTOF_d{ch};TOF [#mus];counts",100000,0,1000))
    hEn_model.append(ROOT.RDF.TH1DModel(f"hEn_d{ch}",f"hEn_d{ch};En [eV];counts",1000,0,100))
    hPulseHeight_model.append(ROOT.RDF.TH1DModel(f"hPulseHeight_d{ch}",f"hPulseHeight_d{ch};Pulse Height;counts",10000,0,10000))
    hEgam_model.append(ROOT.RDF.TH1DModel(f"hEgam_d{ch}",f"hEgam_d{ch};Egam [keV];counts",10000,0,10000*calib[ch]))


for ch in range(numch):
    
    df_ch.append(df.Filter(f"detector == {ch}").Define("Egam",f"PulseHeight*{calib[ch]}"))
    df_ch_pu.append(df_pu.Filter(f"detector == {ch}").Define("Egam",f"PulseHeight*{calib[ch]}"))

    
    hTOF.append(df_ch[ch].Histo1D(hTOF_model[ch],"tof_mus"))
    hEn.append(df_ch[ch].Histo1D(hEn_model[ch],"En"))
    hPulseHeight.append(df_ch[ch].Histo1D(hPulseHeight_model[ch],"PulseHeight"))
    hEgam.append(df_ch[ch].Histo1D(hEgam_model[ch],"Egam"))

    hTOF_pu.append(df_ch_pu[ch].Histo1D(hTOF_model[ch],"tof_mus"))
    hEn_pu.append(df_ch_pu[ch].Histo1D(hEn_model[ch],"En"))
    hPulseHeight_pu.append(df_ch_pu[ch].Histo1D(hPulseHeight_model[ch],"PulseHeight"))
    hEgam_pu.append(df_ch_pu[ch].Histo1D(hEgam_model[ch],"Egam"))


df.Describe().Print()

Dataframe from TChain rawTree in file rawroot/rawroot_run_0014_000.root

Property                Value
--------                -----
Columns in total            9
Columns from defines        2
Event loops run             0
Processing slots            8

Column          Type            Origin
------          ----            ------
Coin            UInt_t          Dataset
En              double          Define
Flags           UInt_t          Dataset
PulseHeight     UShort_t        Dataset
Timestamp       ULong64_t       Dataset
detector        Int_t           Dataset
nTrigger        ULong64_t       Dataset
tof             ULong64_t       Dataset
tof_mus         double          Define

In [13]:
%%time
# hTOF_list[3].Draw()
# hEn_list[3].Draw()
hPulseHeight[1].Draw()
# hEgam_list[1].Draw()
hEgam[1].Draw("same")
ROOT.gROOT.GetListOfCanvases().Draw()
print(f"{df.GetNRuns()} event loop(s)")

1 event loop(s)
CPU times: user 11min 34s, sys: 1.57 s, total: 11min 35s
Wall time: 1min 37s


In [93]:
# hTOF_list[1].Draw()
# ROOT.gROOT.GetListOfCanvases().Draw()

In [118]:
print(f"{df.GetNRuns()} event loop(s)")

1 event loop(s)
